In [40]:
import dash
from dash import dcc, html
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc


import plotly.express as px

import pandas as pd
#datareader has few api to download data from web
import pandas_datareader.data as web
import datetime

In [22]:
start=datetime.datetime(2020,1,1)
end=datetime.date.today()


df=web.DataReader(['GOOGL','FB','PFE','BNTX','MRNA'],'stooq',start=start,end=end)

df=df.stack().reset_index()
df.rename(columns = {'level_0':'Date'}, inplace = True)
df.head()

Attributes,Date,Symbols,Close,High,Low,Open,Volume
0,2020-01-02,GOOGL,68.4340,68.4340,67.3245,67.4205,2.728530e+07
1,2020-01-02,PFE,35.0488,35.2381,34.8126,35.1943,1.749704e+07
2,2020-01-02,BNTX,38.1343,38.1343,35.0241,35.4104,1.410105e+05
3,2020-01-02,MRNA,19.2300,19.8100,18.8800,19.5700,1.229434e+06
4,2020-01-03,GOOGL,68.0760,68.6875,67.3660,67.4000,2.341258e+07


In [35]:
df.to_csv('mystocks.csv',index=False)

In [36]:
app=dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX],
             meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}])

In [42]:
# Layout Section : Bootstrap

app.layout= dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Stock Market Dashboard",
                       className='text-center text-primary, mb-4'),
               width=12)
        
    ]),
    
    
     dbc.Row([
        dbc.Col([
            dcc.Dropdown(id='my-dpdn',multi=False,value='GOOGL',
                        options=[{'label':x, 'value':x}
                                 for x in sorted(df['Symbols'].unique())]),
            
             dcc.Graph(id='line-fig', figure={})
            
        ], # width={'size':5, 'offset':0}
            xs=12, sm=12, md=12, lg=5, xl=5
        
        ),
         
         dbc.Col([
                 dcc.Dropdown(id='my-dpdn2', multi=True, value=['PFE','BNTX'],
                         options=[{'label':x, 'value':x}
                                  for x in sorted(df['Symbols'].unique())],
                         ),
                dcc.Graph(id='line-fig2', figure={})
             
             ], # width={'size':5, 'offset':0}
            xs=12, sm=12, md=12, lg=5, xl=5)
        ], justify='around'),
    
    
    
     dbc.Row([
             dbc.Col([
             html.P("Select Company Stock:",
                   style={'textDecoration':'underline'}),
             dbc.Checklist(id='my-checklist', value=['FB', 'GOOGL', 'AMZN'],
                          options=[{'label':x, 'value':x}
                                   for x in sorted(df['Symbols'].unique())
                                    ], inline=True, labelClassName='text-success'),
             dcc.Graph(id='my-hist',figure={})
             
        
                ], # width={'size':5, 'offset':0}
            xs=12, sm=12, md=12, lg=5, xl=5),
            dbc.Col([
                        dbc.Card(
                            [
                                dbc.CardBody(
                                    html.P(
                                        "We're better together. Help each other out!",
                                        className="card-text")
                                ),
                                dbc.CardImg(
                                    src="https://media.giphy.com/media/iYTXFJofI7I987H92k/giphy-downsized-large.gif",
                                    bottom=True),
                            ],
                            style={"width": "24rem"},
                        )
                 ], # width={'size':5, 'offset':0}
            xs=12, sm=12, md=12, lg=5, xl=5)
          ],align='center')

    
     
])

In [38]:
@app.callback(
    Output('line-fig', 'figure'),
    Input('my-dpdn', 'value')
)
def update_graph(stock_slctd):
    dff = df[df['Symbols']==stock_slctd]
    figln = px.line(dff, x='Date', y='High')
    return figln


# Line chart - multiple
@app.callback(
    Output('line-fig2', 'figure'),
    Input('my-dpdn2', 'value')
)
def update_graph(stock_slctd):
    dff = df[df['Symbols'].isin(stock_slctd)]
    figln2 = px.line(dff, x='Date', y='Open', color='Symbols')
    return figln2


# Histogram
@app.callback(
    Output('my-hist', 'figure'),
    Input('my-checklist', 'value')
)
def update_graph(stock_slctd):
    dff = df[df['Symbols'].isin(stock_slctd)]
    dff = dff[dff['Date']=='2020-12-03']
    fighist = px.histogram(dff, x='Symbols', y='Close')
    return fighist

In [39]:
if __name__=='__main__':
    app.run_server(port=8000)

Dash is running on http://127.0.0.1:8000/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Apr/2023 14:40:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 14:40:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 14:40:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 14:40:43] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 14:40:43] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 14:40:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 14:40:43] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 14:40:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 14:40:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 14:40:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 14:40:50] "POST /_dash-update-component HTTP/1.